In [268]:
import pandas as pd
import numpy as np

In [191]:
# df_link_id = pd.read_csv('link_id.csv', dtype={'ROUTE_ID': str})
# df_visum_link = pd.read_csv('visum_link.csv')
# df_visum_stop = pd.read_csv('visum_stop.csv')

# df_columns = ['$LINEROUTEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'INDEX', 'ISROUTEPOINT', 'NODENO', 'STOPPOINTNO', 'POSTLENGTH', 'ADDVAL']
# df_line_route_items = pd.DataFrame(columns=df_columns)
# df_line_route_items.to_csv('line_route_items.csv', index=False, encoding='utf-8-sig')

# DIRECTIONCODE = '>'
# POSTLENGTH = '0km'
# ADDVAL = 0

# INDEX = 1
# F_NODE = -1
# T_NODE = -1
# link_id = -1
# prev_stop = -1
# prev_node = -1
# prev_link = -1

# for _, l in df_link_id.iterrows():
#     if route_id != l['ROUTE_ID']:
#         df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
#         df_line_route_items = pd.DataFrame(columns=df_columns)
#         INDEX = 1
#         F_NODE = -1
#         T_NODE = -1
    
#     route_id = l['ROUTE_ID']
#     route_name = l['ROUTE_NAME']
#     link_id = l['LINK_ID']
#     link_idx = l['INDEX']
    
#     if prev_link == link_id:
#         continue
        
#     prev_link = link_id
    
#     df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
#     for _, vl in df_vlink.iterrows():
#         df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == vl['LINK_NO']]
#         df_vstop = df_vstop.sort_values(by=['INDEX'])
        
#         for _, vs in df_vstop.iterrows():
#             if INDEX > 1:
#                 break
            
#             df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#             INDEX = INDEX + 1
#             F_NODE = vs['F_NODE']
#             prev_stop = vs['STOP_NO']

KeyboardInterrupt: 

In [267]:
df_link_id = pd.read_csv('link_id.csv', dtype={'ROUTE_ID': str})
df_visum_link = pd.read_csv('visum_link.csv')
df_visum_stop = pd.read_csv('visum_stop.csv')

df_columns = ['$LINEROUTEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'INDEX', 'ISROUTEPOINT', 'NODENO', 'STOPPOINTNO', 'POSTLENGTH', 'ADDVAL']
df_line_route_items = pd.DataFrame(columns=df_columns)
df_line_route_items.to_csv('line_route_items.csv', index=False, encoding='utf-8-sig')

DIRECTIONCODE = '>'
POSTLENGTH = '0km'
ADDVAL = 0

INDEX = 1
START_NODE = -1
NEXT_NODE = -1
prev_route_id = -1
prev_stop = -1
prev_link_no = -1

for i, l in df_link_id.iterrows():
    print(i, '/', len(df_link_id))
    
    route_id = l['ROUTE_ID']
    route_name = l['ROUTE_NAME']
    link_id = l['LINK_ID']
    link_idx = l['INDEX']
    
    if prev_route_id != route_id:
        df_line_route_items.to_csv('line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_line_route_items = pd.DataFrame(columns=df_columns)
        INDEX = 1
        START_NODE = -1
        NEXT_NODE = -1
        prev_stop = -1
        prev_link_no = -1
    
    prev_route_id = route_id
    
    if prev_link == link_id:
        continue
    
    prev_link = link_id
    
    if INDEX == 1:
        df_vlink = df_visum_link[df_visum_link['LINK_ID'] == link_id]
        
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']

            df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
            df_vstop = df_vstop.sort_values(by=['INDEX'])

            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                
                prev_stop = vs['STOP_NO']
                
                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                INDEX = INDEX + 1
                START_NODE = vs['F_NODE']
                NEXT_NODE = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == START_NODE) & (df_visum_link['LINK_NO'] == link_no)].T_NODE.item()
    else:
        df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == NEXT_NODE)]
        
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']
            
            if prev_link_no == link_no:
                continue
            
            prev_link_no = link_no
            
            df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
            INDEX = INDEX + 1
            
            NEXT_NODE = vl['T_NODE']
            
            df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
            df_vstop = df_vstop.sort_values(by=['INDEX'])

            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                
                prev_stop = vs['STOP_NO']
                
                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                INDEX = INDEX + 1
                
        if len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO'])) > 1:
            df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == NEXT_NODE)]
        
            for _, vl in df_vlink.iterrows():
                link_no = vl['LINK_NO']

                if prev_link_no == link_no:
                    continue

                prev_link_no = link_no

                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                INDEX = INDEX + 1

                NEXT_NODE = vl['T_NODE']

                df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
                df_vstop = df_vstop.sort_values(by=['INDEX'])

                for _, vs in df_vstop.iterrows():
                    if prev_stop == vs['STOP_NO']:
                        continue

                    prev_stop = vs['STOP_NO']

                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
                    INDEX = INDEX + 1

            
#         if len(set(df_visum_link[(df_visum_link['LINK_ID'] == link_id)]['LINK_NO'])) > 1:
#             print(df_visum_link[(df_visum_link['LINK_ID'] == link_id)])
#             df_vlink = df_visum_link[(df_visum_link['LINK_ID'] == link_id) & (df_visum_link['F_NODE'] == NEXT_NODE) & (df_visum_link['LINK_NO'] != prev_link_no)]

#             for _, vl in df_vlink.iterrows():
#                 link_no = vl['LINK_NO']
#                 NEXT_NODE = vl['T_NODE']

# #                     if prev_link_no == link_no:
# #                         continue

#                 prev_link_no = link_no

#                 df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, NEXT_NODE, '', POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#                 INDEX = INDEX + 1

#                 df_vstop = df_visum_stop[df_visum_stop['LINK_NO'] == link_no]
#                 df_vstop = df_vstop.sort_values(by=['INDEX'])

#                 for _, vs in df_vstop.iterrows():
#                     if prev_stop == vs['STOP_NO']:
#                         continue

#                     prev_stop = vs['STOP_NO']

#                     df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_columns), ignore_index=True)
#                     INDEX = INDEX + 1

0 / 55598
1 / 55598
2 / 55598
3 / 55598
4 / 55598
5 / 55598
6 / 55598
7 / 55598
8 / 55598
9 / 55598
10 / 55598
11 / 55598
12 / 55598
13 / 55598
14 / 55598
15 / 55598
16 / 55598
17 / 55598
18 / 55598
19 / 55598
20 / 55598
21 / 55598
22 / 55598
23 / 55598
24 / 55598
25 / 55598
26 / 55598
27 / 55598
28 / 55598
29 / 55598
30 / 55598
31 / 55598
32 / 55598
33 / 55598
34 / 55598
35 / 55598
36 / 55598
37 / 55598
38 / 55598
39 / 55598
40 / 55598
41 / 55598
42 / 55598
43 / 55598
44 / 55598
45 / 55598
46 / 55598
47 / 55598
48 / 55598
49 / 55598
50 / 55598
51 / 55598
52 / 55598
53 / 55598
54 / 55598
55 / 55598
56 / 55598
57 / 55598
58 / 55598
59 / 55598
60 / 55598
61 / 55598
62 / 55598
63 / 55598
64 / 55598
65 / 55598
66 / 55598
67 / 55598
68 / 55598
69 / 55598
70 / 55598
71 / 55598
72 / 55598
73 / 55598
74 / 55598
75 / 55598
76 / 55598
77 / 55598
78 / 55598
79 / 55598
80 / 55598
81 / 55598
82 / 55598
83 / 55598
84 / 55598
85 / 55598
86 / 55598
87 / 55598
88 / 55598
89 / 55598
90 / 55598
91 / 5559

712 / 55598
713 / 55598
714 / 55598
715 / 55598
716 / 55598
717 / 55598
718 / 55598
719 / 55598
720 / 55598
721 / 55598
722 / 55598
723 / 55598
724 / 55598
725 / 55598
726 / 55598
727 / 55598
728 / 55598
729 / 55598
730 / 55598
731 / 55598
732 / 55598
733 / 55598
734 / 55598
735 / 55598
736 / 55598
737 / 55598
738 / 55598
739 / 55598
740 / 55598
741 / 55598
742 / 55598
743 / 55598
744 / 55598
745 / 55598
746 / 55598
747 / 55598
748 / 55598
749 / 55598
750 / 55598
751 / 55598
752 / 55598
753 / 55598
754 / 55598
755 / 55598
756 / 55598
757 / 55598
758 / 55598
759 / 55598
760 / 55598
761 / 55598
762 / 55598
763 / 55598
764 / 55598
765 / 55598
766 / 55598
767 / 55598
768 / 55598
769 / 55598
770 / 55598
771 / 55598
772 / 55598
773 / 55598
774 / 55598
775 / 55598
776 / 55598
777 / 55598
778 / 55598
779 / 55598
780 / 55598
781 / 55598
782 / 55598
783 / 55598
784 / 55598
785 / 55598
786 / 55598
787 / 55598
788 / 55598
789 / 55598
790 / 55598
791 / 55598
792 / 55598
793 / 55598
794 / 55598
795 

1393 / 55598
1394 / 55598
1395 / 55598
1396 / 55598
1397 / 55598
1398 / 55598
1399 / 55598
1400 / 55598
1401 / 55598
1402 / 55598
1403 / 55598
1404 / 55598
1405 / 55598
1406 / 55598
1407 / 55598
1408 / 55598
1409 / 55598
1410 / 55598
1411 / 55598
1412 / 55598
1413 / 55598
1414 / 55598
1415 / 55598
1416 / 55598
1417 / 55598
1418 / 55598
1419 / 55598
1420 / 55598
1421 / 55598
1422 / 55598
1423 / 55598
1424 / 55598
1425 / 55598
1426 / 55598
1427 / 55598
1428 / 55598
1429 / 55598
1430 / 55598
1431 / 55598
1432 / 55598
1433 / 55598
1434 / 55598
1435 / 55598
1436 / 55598
1437 / 55598
1438 / 55598
1439 / 55598
1440 / 55598
1441 / 55598
1442 / 55598
1443 / 55598
1444 / 55598
1445 / 55598
1446 / 55598
1447 / 55598
1448 / 55598
1449 / 55598
1450 / 55598
1451 / 55598
1452 / 55598
1453 / 55598
1454 / 55598
1455 / 55598
1456 / 55598
1457 / 55598
1458 / 55598
1459 / 55598
1460 / 55598
1461 / 55598
1462 / 55598
1463 / 55598
1464 / 55598
1465 / 55598
1466 / 55598
1467 / 55598
1468 / 55598
1469 / 55598

2050 / 55598
2051 / 55598
2052 / 55598
2053 / 55598
2054 / 55598
2055 / 55598
2056 / 55598
2057 / 55598
2058 / 55598
2059 / 55598
2060 / 55598
2061 / 55598
2062 / 55598
2063 / 55598
2064 / 55598
2065 / 55598
2066 / 55598
2067 / 55598
2068 / 55598
2069 / 55598
2070 / 55598
2071 / 55598
2072 / 55598
2073 / 55598
2074 / 55598
2075 / 55598
2076 / 55598
2077 / 55598
2078 / 55598
2079 / 55598
2080 / 55598
2081 / 55598
2082 / 55598
2083 / 55598
2084 / 55598
2085 / 55598
2086 / 55598
2087 / 55598
2088 / 55598
2089 / 55598
2090 / 55598
2091 / 55598
2092 / 55598
2093 / 55598
2094 / 55598
2095 / 55598
2096 / 55598
2097 / 55598
2098 / 55598
2099 / 55598
2100 / 55598
2101 / 55598
2102 / 55598
2103 / 55598
2104 / 55598
2105 / 55598
2106 / 55598
2107 / 55598
2108 / 55598
2109 / 55598
2110 / 55598
2111 / 55598
2112 / 55598
2113 / 55598
2114 / 55598
2115 / 55598
2116 / 55598
2117 / 55598
2118 / 55598
2119 / 55598
2120 / 55598
2121 / 55598
2122 / 55598
2123 / 55598
2124 / 55598
2125 / 55598
2126 / 55598

2680 / 55598
2681 / 55598
2682 / 55598
2683 / 55598
2684 / 55598
2685 / 55598
2686 / 55598
2687 / 55598
2688 / 55598
2689 / 55598
2690 / 55598
2691 / 55598
2692 / 55598
2693 / 55598
2694 / 55598
2695 / 55598
2696 / 55598
2697 / 55598
2698 / 55598
2699 / 55598
2700 / 55598
2701 / 55598
2702 / 55598
2703 / 55598
2704 / 55598
2705 / 55598
2706 / 55598
2707 / 55598
2708 / 55598
2709 / 55598
2710 / 55598
2711 / 55598
2712 / 55598
2713 / 55598
2714 / 55598
2715 / 55598
2716 / 55598
2717 / 55598
2718 / 55598
2719 / 55598
2720 / 55598
2721 / 55598
2722 / 55598
2723 / 55598
2724 / 55598
2725 / 55598
2726 / 55598
2727 / 55598
2728 / 55598
2729 / 55598
2730 / 55598
2731 / 55598
2732 / 55598
2733 / 55598
2734 / 55598
2735 / 55598
2736 / 55598
2737 / 55598
2738 / 55598
2739 / 55598
2740 / 55598
2741 / 55598
2742 / 55598
2743 / 55598
2744 / 55598
2745 / 55598
2746 / 55598
2747 / 55598
2748 / 55598
2749 / 55598
2750 / 55598
2751 / 55598
2752 / 55598
2753 / 55598
2754 / 55598
2755 / 55598
2756 / 55598

...

52688 / 55598
52689 / 55598
52690 / 55598
52691 / 55598
52692 / 55598
52693 / 55598
52694 / 55598
52695 / 55598
52696 / 55598
52697 / 55598
52698 / 55598
52699 / 55598
52700 / 55598
52701 / 55598
52702 / 55598
52703 / 55598
52704 / 55598
52705 / 55598
52706 / 55598
52707 / 55598
52708 / 55598
52709 / 55598
52710 / 55598
52711 / 55598
52712 / 55598
52713 / 55598
52714 / 55598
52715 / 55598
52716 / 55598
52717 / 55598
52718 / 55598
52719 / 55598
52720 / 55598
52721 / 55598
52722 / 55598
52723 / 55598
52724 / 55598
52725 / 55598
52726 / 55598
52727 / 55598
52728 / 55598
52729 / 55598
52730 / 55598
52731 / 55598
52732 / 55598
52733 / 55598
52734 / 55598
52735 / 55598
52736 / 55598
52737 / 55598
52738 / 55598
52739 / 55598
52740 / 55598
52741 / 55598
52742 / 55598
52743 / 55598
52744 / 55598
52745 / 55598
52746 / 55598
52747 / 55598
52748 / 55598
52749 / 55598
52750 / 55598
52751 / 55598
52752 / 55598
52753 / 55598
52754 / 55598
52755 / 55598
52756 / 55598
52757 / 55598
52758 / 55598
52759 

53288 / 55598
53289 / 55598
53290 / 55598
53291 / 55598
53292 / 55598
53293 / 55598
53294 / 55598
53295 / 55598
53296 / 55598
53297 / 55598
53298 / 55598
53299 / 55598
53300 / 55598
53301 / 55598
53302 / 55598
53303 / 55598
53304 / 55598
53305 / 55598
53306 / 55598
53307 / 55598
53308 / 55598
53309 / 55598
53310 / 55598
53311 / 55598
53312 / 55598
53313 / 55598
53314 / 55598
53315 / 55598
53316 / 55598
53317 / 55598
53318 / 55598
53319 / 55598
53320 / 55598
53321 / 55598
53322 / 55598
53323 / 55598
53324 / 55598
53325 / 55598
53326 / 55598
53327 / 55598
53328 / 55598
53329 / 55598
53330 / 55598
53331 / 55598
53332 / 55598
53333 / 55598
53334 / 55598
53335 / 55598
53336 / 55598
53337 / 55598
53338 / 55598
53339 / 55598
53340 / 55598
53341 / 55598
53342 / 55598
53343 / 55598
53344 / 55598
53345 / 55598
53346 / 55598
53347 / 55598
53348 / 55598
53349 / 55598
53350 / 55598
53351 / 55598
53352 / 55598
53353 / 55598
53354 / 55598
53355 / 55598
53356 / 55598
53357 / 55598
53358 / 55598
53359 

53904 / 55598
53905 / 55598
53906 / 55598
53907 / 55598
53908 / 55598
53909 / 55598
53910 / 55598
53911 / 55598
53912 / 55598
53913 / 55598
53914 / 55598
53915 / 55598
53916 / 55598
53917 / 55598
53918 / 55598
53919 / 55598
53920 / 55598
53921 / 55598
53922 / 55598
53923 / 55598
53924 / 55598
53925 / 55598
53926 / 55598
53927 / 55598
53928 / 55598
53929 / 55598
53930 / 55598
53931 / 55598
53932 / 55598
53933 / 55598
53934 / 55598
53935 / 55598
53936 / 55598
53937 / 55598
53938 / 55598
53939 / 55598
53940 / 55598
53941 / 55598
53942 / 55598
53943 / 55598
53944 / 55598
53945 / 55598
53946 / 55598
53947 / 55598
53948 / 55598
53949 / 55598
53950 / 55598
53951 / 55598
53952 / 55598
53953 / 55598
53954 / 55598
53955 / 55598
53956 / 55598
53957 / 55598
53958 / 55598
53959 / 55598
53960 / 55598
53961 / 55598
53962 / 55598
53963 / 55598
53964 / 55598
53965 / 55598
53966 / 55598
53967 / 55598
53968 / 55598
53969 / 55598
53970 / 55598
53971 / 55598
53972 / 55598
53973 / 55598
53974 / 55598
53975 

54520 / 55598
54521 / 55598
54522 / 55598
54523 / 55598
54524 / 55598
54525 / 55598
54526 / 55598
54527 / 55598
54528 / 55598
54529 / 55598
54530 / 55598
54531 / 55598
54532 / 55598
54533 / 55598
54534 / 55598
54535 / 55598
54536 / 55598
54537 / 55598
54538 / 55598
54539 / 55598
54540 / 55598
54541 / 55598
54542 / 55598
54543 / 55598
54544 / 55598
54545 / 55598
54546 / 55598
54547 / 55598
54548 / 55598
54549 / 55598
54550 / 55598
54551 / 55598
54552 / 55598
54553 / 55598
54554 / 55598
54555 / 55598
54556 / 55598
54557 / 55598
54558 / 55598
54559 / 55598
54560 / 55598
54561 / 55598
54562 / 55598
54563 / 55598
54564 / 55598
54565 / 55598
54566 / 55598
54567 / 55598
54568 / 55598
54569 / 55598
54570 / 55598
54571 / 55598
54572 / 55598
54573 / 55598
54574 / 55598
54575 / 55598
54576 / 55598
54577 / 55598
54578 / 55598
54579 / 55598
54580 / 55598
54581 / 55598
54582 / 55598
54583 / 55598
54584 / 55598
54585 / 55598
54586 / 55598
54587 / 55598
54588 / 55598
54589 / 55598
54590 / 55598
54591 

55135 / 55598
55136 / 55598
55137 / 55598
55138 / 55598
55139 / 55598
55140 / 55598
55141 / 55598
55142 / 55598
55143 / 55598
55144 / 55598
55145 / 55598
55146 / 55598
55147 / 55598
55148 / 55598
55149 / 55598
55150 / 55598
55151 / 55598
55152 / 55598
55153 / 55598
55154 / 55598
55155 / 55598
55156 / 55598
55157 / 55598
55158 / 55598
55159 / 55598
55160 / 55598
55161 / 55598
55162 / 55598
55163 / 55598
55164 / 55598
55165 / 55598
55166 / 55598
55167 / 55598
55168 / 55598
55169 / 55598
55170 / 55598
55171 / 55598
55172 / 55598
55173 / 55598
55174 / 55598
55175 / 55598
55176 / 55598
55177 / 55598
55178 / 55598
55179 / 55598
55180 / 55598
55181 / 55598
55182 / 55598
55183 / 55598
55184 / 55598
55185 / 55598
55186 / 55598
55187 / 55598
55188 / 55598
55189 / 55598
55190 / 55598
55191 / 55598
55192 / 55598
55193 / 55598
55194 / 55598
55195 / 55598
55196 / 55598
55197 / 55598
55198 / 55598
55199 / 55598
55200 / 55598
55201 / 55598
55202 / 55598
55203 / 55598
55204 / 55598
55205 / 55598
55206 